In [ ]:
import tensorflow as tf

# Seems definitely wrong
class WeightDecayModel(tf.keras.Model):
    def __init__(self, weight_decay, *args, **kwargs):
        super(WeightDecayModel, self).__init__(*args, **kwargs)
        self.weight_decay = weight_decay
    
    def call(self, inputs, training=False):
        if training:
            l1_penalty = tf.add_n(tf.reduce_sum(x) for x in self.trainable_variables)
            self.add_loss(l1_penalty * self.weight_decay)
        return super(WeightDecayModel, self).call(inputs, training)

In [ ]:
# Seems more plausible
class WeightDecayModel(tf.keras.Model):
    def __init__(self, weight_decay, *args, **kwargs):
        super(WeightDecayModel, self).__init__(*args, **kwargs)
        self.weight_decay = tf.convert_to_tensor(weight_decay)

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        for var in trainable_vars:
            # Why am I multiplying by learning rate?
            # Because 
            var.assign_sub(self.optimizer.lr * self.weight_decay * var)
        return {"loss": loss}